# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
data = spark.sql("SELECT * FROM smsspamcollection")

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [0]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

## Clean and Prepare the Data

** Create a new length feature: **

In [0]:
from pyspark.sql.functions import length

In [0]:
data = data.withColumn('length',length(data['text']))

In [0]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham|71.48663212435233|
 spam|138.6706827309237|
+-----+-----------------+

## Feature Transformations

In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data)

In [0]:
clean_data = cleaner.transform(data)

In [0]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
class| text|length|label| token_text| stop_tokens| c_vec| tf_idf| features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 ham|Go until jurong p...| 111| 0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
 ham|Ok lar... Joking ...| 29| 0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
 spam|Free entry in 2 a...| 155| 1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
 ham|U dun say so earl...| 49| 0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,70,80,1...|
 ham|Nah I don't think...| 61| 0.0|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|(13424,[36,134,31...|
 spam|FreeMsg Hey there...| 147| 1.0|[freemsg, hey, th...|[freemsg, hey, da...|(13423,[10,60,139...|(13423,[10,60,139...|(13424,[10,60,139...|
 ham|Even my brother i...| 77| 0.0|[even, my, brothe...|[even, brother, l...|(13423,[10,53,103...|(13423,[10,53,103...|(13424,[10,53,103...|
 ham|As per your reque...| 160| 0.0|[as, per, your, r...|[per, request, 'm...|(13423,[125,184,4...|(13423,[125,184,4...|(13424,[125,184,4...|
 spam|WINNER!! As a val...| 157| 1.0|[winner!!, as, a,...|[winner!!, valued...|(13423,[1,47,118,...|(13423,[1,47,118,...|(13424,[1,47,118,...|
 spam|Had your mobile 1...| 154| 1.0|[had, your, mobil...|[mobile, 11, mont...|(13423,[0,1,13,27...|(13423,[0,1,13,27...|(13424,[0,1,13,27...|
 ham|I'm gonna be home...| 109| 0.0|[i'm, gonna, be, ...|[gonna, home, soo...|(13423,[18,43,120...|(13423,[18,43,120...|(13424,[18,43,120...|
 spam|SIX chances to wi...| 136| 1.0|[six, chances, to...|[six, chances, wi...|(13423,[8,17,37,8...|(13423,[8,17,37,8...|(13424,[8,17,37,8...|
 spam|URGENT! You have ...| 155| 1.0|[urgent!, you, ha...|[urgent!, won, 1,...|(13423,[13,30,47,...|(13423,[13,30,47,...|(13424,[13,30,47,...|
 ham|I've been searchi...| 196| 0.0|[i've, been, sear...|[searching, right...|(13423,[39,96,217...|(13423,[39,96,217...|(13424,[39,96,217...|
 ham|I HAVE A DATE ON ...| 35| 0.0|[i, have, a, date...|[date, sunday, wi...|(13423,[552,1697,...|(13423,[552,1697,...|(13424,[552,1697,...|
 spam|XXXMobileMovieClu...| 149| 1.0|[xxxmobilemoviecl...|[xxxmobilemoviecl...|(13423,[30,109,11...|(13423,[30,109,11...|(13424,[30,109,11...|
 ham|Oh k...i'm watchi...| 26| 0.0|[oh, k...i'm, wat...|[oh, k...i'm, wat...|(13423,[82,214,47...|(13423,[82,214,47...|(13424,[82,214,47...|
 ham|Eh u remember how...| 81| 0.0|[eh, u, remember,...|[eh, u, remember,...|(13423,[0,2,49,13...|(13423,[0,2,49,13...|(13424,[0,2,49,13...|
 ham|Fine if thats th...| 56| 0.0|[fine, if, thats...|[fine, thats, wa...|(13423,[0,74,105,...|(13423,[0,74,105,...|(13424,[0,74,105,...|
 spam|England v Macedon...| 155| 1.0|[england, v, mace...|[england, v, mace...|(13423,[4,30,33,5...|(13423,[4,30,33,5...|(13424,[4,30,33,5...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

### Training and Evaluation!

In [0]:
clean_data = clean_data.select(['label','features'])

In [0]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [0]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [0]:
spam_predictor = nb.fit(training)

In [0]:
data.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [0]:
test_results = spam_predictor.transform(testing)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,7,8...|[-798.06170804950...|[1.0,1.6619720874...| 0.0|
 0.0|(13424,[0,1,2,13,...|[-610.09663138886...|[1.0,4.8349752657...| 0.0|
 0.0|(13424,[0,1,2,41,...|[-1065.8211683618...|[1.0,5.7834240982...| 0.0|
 0.0|(13424,[0,1,3,9,1...|[-573.40426624392...|[0.99999999999937...| 0.0|
 0.0|(13424,[0,1,4,50,...|[-836.19126642316...|[1.0,1.6543516130...| 0.0|
 0.0|(13424,[0,1,5,20,...|[-804.71116955433...|[1.0,1.7211425465...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-875.81716107553...|[1.0,1.1848606169...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-1186.0948979754...|[1.0,8.5050874234...| 0.0|
 0.0|(13424,[0,1,14,31...|[-220.77005615409...|[1.0,3.3154702935...| 0.0|
 0.0|(13424,[0,1,15,20...|[-695.55493594734...|[1.0,1.9025652045...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1295.4972036133...|[1.0,2.8312077473...| 0.0|
 0.0|(13424,[0,1,31,43...|[-343.40096248590...|[1.0,7.9049605935...| 0.0|
 0.0|(13424,[0,2,3,4,6...|[-1291.2477864733...|[1.0,2.7532175856...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3297.3205972224...|[1.0,2.1505725225...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2743.5623984775...|[1.0,2.4555471984...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2743.5623984775...|[1.0,2.4555471984...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-578.88119100564...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1125.1427074408...|[1.0,2.1272873715...| 0.0|
 0.0|(13424,[0,2,4,25,...|[-419.88156174036...|[1.0,3.3957062853...| 0.0|
 0.0|(13424,[0,2,4,128...|[-658.87893419116...|[1.0,1.1453174978...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9244108719429434